In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from langchain.chat_models import ChatOpenAI

c:\Users\91984\Desktop\ai\langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    encode_kwargs={"normalize_embeddings": True},  # for cosine similarity
)

C:\Users\sagar\AppData\Local\Temp\ipykernel_24220\3253216364.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [4]:
# Test the embeddings to make sure they work
test_text = "Hello Nagaraj, how are you?"
test_embedding = embeddings.embed_query(test_text)
print(len(test_embedding))
print(test_embedding[:5])



768
[0.03317498415708542, 0.007324717938899994, -0.007353310473263264, 0.020806776359677315, 0.04681262746453285]


In [5]:
from transformers import AutoTokenizer

# Load the tokenizer for MiniLM
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Your text
text = "Hello Nagaraj, how are you?"

# Tokenize
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.encode(text, add_special_tokens=True)

# Print results
print("Tokens:", tokens)
print("Token IDs:", token_ids)

Tokens: ['hello', 'nagar', '##aj', ',', 'how', 'are', 'you', '?']
Token IDs: [101, 7592, 16395, 13006, 1010, 2129, 2024, 2017, 1029, 102]


In [6]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA
file = 'revenue.csv'


In [7]:
loader = CSVLoader(file_path=file)
docs = loader.load()
#print(docs)

db = DocArrayInMemorySearch.from_documents(docs, embeddings)


c:\Users\91984\Desktop\ai\langchain\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
print(docs)


[Document(metadata={'source': 'revenue.csv', 'row': 0}, page_content='Date: 06/07/2025\nNo of Orders: 1\nOrder details: lotus*7, sugandaraja hara*1\nCost of procurement: 85\nDelivery Cost: 120\nTotal Cost: 205\nTotal Revenue: 205\nGross Profit: 0\nContact: 7349580261\nArea: elect. city, 2nd \nphase\nPincode: 560100\n: '), Document(metadata={'source': 'revenue.csv', 'row': 1}, page_content='Date: 07/07/2025\nNo of Orders: 1\nOrder details: lotus*2, sooji mallige 100gm, sugandaraja \n250gm,\nCost of procurement: 120\nDelivery Cost: 50\nTotal Cost: 170\nTotal Revenue: 105\nGross Profit: -65\nContact: 9036601445\nArea: JP Nagar, 4th\n phase\nPincode: 560078\n: '), Document(metadata={'source': 'revenue.csv', 'row': 2}, page_content='Date: 08/07/2025\nNo of Orders: 1\nOrder details: Kanakambara maale*2, sugandaraja 1kg\nCost of procurement: 70\nDelivery Cost: 50\nTotal Cost: 120\nTotal Revenue: 200\nGross Profit: 80\nContact: 9980903816\nArea: JP Nagar, 8th\n phase\nPincode: 560062\n: '), Do

In [9]:
#query = "order details of 27/07/2025?"
#results = db.similarity_search(query, k=30) #default is 4

In [10]:
#len(results)
#results[0]

In [11]:
llm = ChatOpenAI(
    temperature=0.0,
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    model="openai/gpt-oss-20b:free",
)

C:\Users\91984\AppData\Local\Temp\ipykernel_4056\3317274095.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [12]:
retriever = db.as_retriever(search_kwargs={"k": 30})

In [13]:
#qdocs = "".join([results[i].page_content for i in range(len(results))])

In [14]:
#response = llm.call_as_llm(f"{qdocs} Question: what is the order on 27/07/2025? analyse the order and give short summary") 
#print(response)

In [15]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [17]:
query =  "analyse any five orders and give a short summary"

In [18]:
response = qa_stuff.run(query)
print(response)

C:\Users\91984\AppData\Local\Temp\ipykernel_4056\123273638.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.
**Order 1 – 01 Aug 2025**  
- **Items**: Mallige dindu × 2 (only item listed)  
- **Cost of procurement**: ₹200  
- **Delivery cost**: ₹50  
- **Total cost**: ₹250  
- **Revenue**: ₹449  
- **Gross profit**: ₹199  
- **Contact**: 997 297 6523  
- **Area**: Not specified (no address or pincode given)  

**Order 2 – 25 Aug 2025**  
- **Items**: A mix of flower arrangements (Mallige Dindu, Kanakambara maale, Spatik maale, Sevanthige yellow maale, etc.) – 12 distinct items listed.  
- **Cost of procurement**: ₹2,060  
- **Delivery cost**: ₹70  
- **Total cost**: ₹2,130  
- **Revenue**: ₹3,050  
- **Gross profit**: ₹920  
- **Contact**: 99452 68993  
- **Area**: Shantinagara, Pincode 560027  

**Order 3 – 04 Sep 2025**  
- **Items**: A large bouquet of mixed flowers (3 Haras, 166 Nandi Vattu, Mallige dindu × 5, lotus × 5, classic × 3, etc.) – 20+ items.  
- **Cost of procurement**: ₹6,170  
- **Delivery cost**: ₹220  
- **Total cost**